# Install environment

In [ ]:
!pip install torch==1.7.0+cu110 numpy dill tqdm torchtext==0.8.0 tensorboard matplotlib scipy click==7.1.2 allennlp==2.4.0 -f https://download.pytorch.org/whl/torch_stable.html

# Download model and dataset
In this section, we loaded one pretrained transformer model. 

In [1]:
!gdown --fuzzy https://drive.google.com/file/d/13Q-dL0G2UDFSqZQBV2XESPSsQhXaNKDl/view?usp=sharing
!gdown --fuzzy https://drive.google.com/file/d/1Lq0qbGauACbjGvXnsOkE2qVD8t026wLi/view?usp=sharing
!gdown --fuzzy https://drive.google.com/file/d/1CeEJ5DaGlpEF7rsY8VYrpuUjADX2w4DX/view?usp=sharing
!gdown --fuzzy https://drive.google.com/file/d/1-LKZhx27toRBYOakZuhmmVJAM8vL4eRT/view?usp=sharing

Downloading...
From: https://drive.google.com/uc?id=13Q-dL0G2UDFSqZQBV2XESPSsQhXaNKDl
To: /mnt/ufs18/home-107/jiajingh/Transformer.zip
100%|████████████████████████████████████████| 160k/160k [00:00<00:00, 23.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Lq0qbGauACbjGvXnsOkE2qVD8t026wLi
To: /mnt/ufs18/home-107/jiajingh/checkpoint.zip
100%|█████████████████████████████████████████| 849M/849M [00:07<00:00, 116MB/s]
Downloading...
From: https://drive.google.com/uc?id=1CeEJ5DaGlpEF7rsY8VYrpuUjADX2w4DX
To: /mnt/ufs18/home-107/jiajingh/test_site_map.json
100%|███████████████████████████████████████| 35.0M/35.0M [00:00<00:00, 108MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-LKZhx27toRBYOakZuhmmVJAM8vL4eRT
To: /mnt/ufs18/home-107/jiajingh/test.tsv
100%|██████████████████████████████████████| 22.7M/22.7M [00:00<00:00, 82.2MB/s]


In [3]:
!rm -r __MACOSX/
!rm -r checkpoint
!rm -r normal/
!rm -r Transformer/
!unzip -q Transformer.zip
!unzip -q checkpoint.zip
%mkdir models/
%mkdir normal/
%ls
%mv checkpoint normal/checkpoints

rm: cannot remove ‘__MACOSX/’: No such file or directory
rm: cannot remove ‘checkpoint’: No such file or directory
rm: cannot remove ‘Transformer/’: No such file or directory


In [4]:
%mkdir -p outputs/gradient-targeting

In [5]:
!pip install scipy
from scipy import spatial

# Data preprocessing
To save the excuting time, we subsampled 100 samples from the whole dataset.

In [21]:
import os
import re
import csv
import sys
import json
import numpy as np
csv.field_size_limit(sys.maxsize)
ID_MAP = {}
with open("test.tsv", 'r') as identity_tsv:
  reader = csv.reader(
    (x.replace('\0', '') for x in identity_tsv),
    delimiter='\t', quoting=csv.QUOTE_NONE
  )
  next(reader, None)
  for line in reader:
    ID_MAP[line[0]] = (line[1], line[2])
print("  + Loaded {} samples".format(len(ID_MAP)))

M = 100
idx_lst = list(range(M))
idx_lst = np.random.choice(np.array(idx_lst),M,replace=False)
print(idx_lst)
cnt = 0
with open("test.tsv", "w") as out_f:
  out_f.write('index\tsrc\ttgt\n')
  for key in ID_MAP.keys():
    if cnt in idx_lst:
        row = [ key,ID_MAP[key][0], ID_MAP[key][1] ]
        out_f.write('{}\n'.format('\t'.join(row)))
        cnt+=1
    else:
        break

  + Loaded 19999 samples
[25 85 32 66 57 77 16 19 15 37 62 60 93 99 64 91 27 76 39 52 79 82 38 96
 45 12 21 28 55 84 46 94 20 26 23 88 54  0 70  2 74 73 98 47 30 86  1 61
 68 33  8 78 48 69 36 51  6 71 13 17 56 89 50 29 72 31 95 53 18  7  5 90
  9 14 11 63 75 67 83 35 87  4 59 42 40 24  3 97 80 34 92 41 81 49 65 22
 58 10 43 44]


In [22]:

import os
import re
import csv
import sys
import tqdm
import json

def handle_replacement_tokens(line):
  new_line = line
  uniques = set()
  for match in re.compile('replaceme\d+').findall(line):
    uniques.add(match.strip())
  uniques = list(uniques)
  uniques.sort()
  uniques.reverse()
  for match in uniques:
    replaced = match.replace("replaceme", "@R_") + '@'
    new_line = new_line.replace(match, replaced)
  return new_line

csv.field_size_limit(sys.maxsize)

ID_MAP = {}
TRANSFORMS = ['transforms.Combined']
print("Loading identity transform...")
with open("test.tsv", 'r') as identity_tsv:
  reader = csv.reader(
    (x.replace('\0', '') for x in identity_tsv),
    delimiter='\t', quoting=csv.QUOTE_NONE
  )
  next(reader, None)
  for line in reader:
    ID_MAP[line[0]] = (line[1], line[2])
print("  + Loaded {} samples".format(len(ID_MAP)))

print("Loading transformed samples...")
TRANSFORMED = {}
for transform_name in TRANSFORMS:
  TRANSFORMED[transform_name] = {}
  with open("test.tsv", 'r') as current_tsv:
    reader = csv.reader(
      (x.replace('\0', '') for x in current_tsv),
      delimiter='\t', quoting=csv.QUOTE_NONE
    )
    next(reader, None)
    for line in reader:
      TRANSFORMED[transform_name][line[0]] = handle_replacement_tokens(line[1])
  print("  + Loaded {} samples from '{}'".format(
    len(TRANSFORMED[transform_name]), transform_name
  ))

print("Writing adv. {}ing samples...".format(sys.argv[1]))
with open("outputs/test.tsv", "w") as out_f:
  out_f.write('index\tsrc\ttgt\t{}\n'.format(
    '\t'.join([ 
      '{}'.format(i) for i in TRANSFORMS
    ])
  ))

  idx_to_fname = {}
  index = 0

  for key in tqdm.tqdm(ID_MAP.keys(), desc="  + Progress"):
    row = [ ID_MAP[key][0], ID_MAP[key][1] ]
    for transform_name in TRANSFORMS:
      if key in TRANSFORMED[transform_name]:
        row.append(TRANSFORMED[transform_name][key])
      else:
        row.append(ID_MAP[key][0])
    out_f.write('{}\t{}\n'.format(index, '\t'.join(row)))
    idx_to_fname[index] = key
    index += 1
with open('outputs/test_idx_to_fname.json', 'w') as f:
  json.dump(idx_to_fname, f)
print("  + Adversarial testing file generation complete!")


  + Progress: 100%|██████████| 100/100 [00:00<00:00, 231985.84it/s]

Loading identity transform...
  + Loaded 100 samples
Loading transformed samples...
  + Loaded 100 samples from 'transforms.Combined'
Writing adv. -fing samples...
  + Adversarial testing file generation complete!


# Evaluation on clean examples


## Clean examples

In [35]:
from scipy import spatial
!python ./Transformer/gradient_attack.py \
    --data_path ./outputs/test.tsv \
    --expt_dir "normal" \
    --load_checkpoint "Best_F1" \
    --save_path ./outputs/targets-test.json \
    --n_alt_iters 2 \
    --z_init 1 \
    --batch_size 32 \
    --u_pgd_epochs 0 \
    --z_epsilon 0 \
    --attack_version 2 \
    --u_learning_rate 0.5 \
    --z_learning_rate 0.5 \
    --smoothing_param 0.01 \
	--vocab_to_use 1 \
    --teacher_ratio 

!python ./Transformer/replace_tokens.py \
      --source_data_path ./outputs/test.tsv \
      --dest_data_path ./outputs/gradient-targeting/test.tsv \
      --mapping_json ./outputs/targets-test-gradient.json
  

mkdir: cannot create directory ‘models/’: File exists
mkdir: cannot create directory ‘normal/’: File exists
anaconda3/                         new_data.jsonl.gz   test.tsv
Anaconda3-2021.05-Linux-x86_64.sh  normal/             test.txt
Attack.ipynb                       ondemand/           Tony/
checkpoint.zip                     optimal.png         train.jsonl
clean.png                          outputs/            train.jsonl.gz
data.jsonl                         preprocess.ipynb    train.txt
Documents/                         programs.tar.gz     Transformer/
environment.yml                    random.png          Transformer.zip
__MACOSX/                          test.jsonl          valid.jsonl
models/                            test.jsonl.gz       valid.jsonl.gz
new_data.jsonl                     test_site_map.json  valid.txt
mv: cannot stat ‘checkpoint’: No such file or directory
/mnt/home/jiajingh/anaconda3/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: Sourc

## Evaluation results on the clean examples

In [24]:
!python3 ./Transformer/evaluate.py \
  --data_path ./outputs/gradient-targeting/test.tsv \
  --expt_dir "normal" \
  --output_dir /outputs \
  --load_checkpoint "Best_F1" \
  --src_field_name "transforms.Combined" --save 

/mnt/home/jiajingh/anaconda3/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: SourceField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/mnt/home/jiajingh/anaconda3/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: TargetField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/mnt/home/jiajingh/anaconda3/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: TransSourceField class will be retired soon and m

# Adversarial examples generation
In this section, you can utilize two version of attacks. 
1. Random select sites (where should be attacked in the sentence), perturb those selected sites.
2. Co-optimize sites and tokens to generate attack. 

## Random site-selection + optimal site-perturbation

In [29]:
%mkdir models/
%mkdir normal/
%mv checkpoint normal/checkpoints
!python3 ./Transformer/gradient_attack.py \
    --data_path ./outputs/test.tsv \
    --expt_dir "normal" \
    --load_checkpoint "Best_F1" \
    --save_path ./outputs/targets-test.json \
    --n_alt_iters 2 \
    --z_init 1 \
    --batch_size 16 \
    --u_pgd_epochs 3 \
    --z_epsilon 1 \
    --attack_version 2 \
    --u_learning_rate 0.5 \
    --z_learning_rate 0.5 \
    --smoothing_param 0.01 \
	--vocab_to_use 1 \
    --teacher_ratio \
    --u_optim
!python3 ./Transformer/replace_tokens.py \
      --source_data_path ./outputs/test.tsv \
      --dest_data_path ./outputs/gradient-targeting/test.tsv \
      --mapping_json ./outputs/targets-test-gradient.json

mkdir: cannot create directory ‘models/’: File exists
mkdir: cannot create directory ‘normal/’: File exists
mv: cannot stat ‘checkpoint’: No such file or directory
/mnt/home/jiajingh/anaconda3/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: SourceField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/mnt/home/jiajingh/anaconda3/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: TargetField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__

## Evaluation results

In [30]:
!python3 ./Transformer/evaluate.py \
  --data_path ./outputs/gradient-targeting/test.tsv \
  --expt_dir "normal" \
  --output_dir ./outputs \
  --load_checkpoint "Best_F1" \
  --src_field_name "transforms.Combined" --save 

/mnt/home/jiajingh/anaconda3/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: SourceField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/mnt/home/jiajingh/anaconda3/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: TargetField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/mnt/home/jiajingh/anaconda3/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: TransSourceField class will be retired soon and m

## optimal site-selection + optimal site-perturbation

In [29]:
%mkdir models/
%mkdir normal/
%mv checkpoint normal/checkpoints
!python3 ./Transformer/gradient_attack.py \
    --data_path ./outputs/test.tsv \
    --expt_dir "normal" \
    --load_checkpoint "Best_F1" \
    --save_path ./outputs/targets-test.json \
    --n_alt_iters 2 \
    --z_init 1 \
    --z_epsilon 1 \
    --batch_size 16 \
    --u_pgd_epochs 3 \
    --attack_version 2 \
    --u_learning_rate 0.5 \
    --z_learning_rate 0.5 \
    --smoothing_param 0.01 \
	--vocab_to_use 1 \
    --teacher_ratio \
    --u_optim \
    --z_optim
!python3 ./Transformer/replace_tokens.py \
      --source_data_path ./outputs/test.tsv \
      --dest_data_path ./outputs/gradient-targeting/test.tsv \
      --mapping_json ./outputs/targets-test-gradient.json

mkdir: cannot create directory ‘models/’: File exists
mkdir: cannot create directory ‘normal/’: File exists
mv: cannot stat ‘checkpoint’: No such file or directory
/mnt/home/jiajingh/anaconda3/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: SourceField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/mnt/home/jiajingh/anaconda3/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: TargetField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__

## Evaluation results


In [30]:
!python3 ./Transformer/evaluate.py \
  --data_path ./outputs/gradient-targeting/test.tsv \
  --expt_dir "normal" \
  --output_dir ./outputs \
  --load_checkpoint "Best_F1" \
  --src_field_name "transforms.Combined" --save 

/mnt/home/jiajingh/anaconda3/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: SourceField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/mnt/home/jiajingh/anaconda3/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: TargetField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/mnt/home/jiajingh/anaconda3/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: TransSourceField class will be retired soon and m

# Visualization
In this section we provide visualization results for adversarial attacks and clean examples.

## clean example
![alt text](clean.png "Title")
## random site-selection + optimal site-perturbation
![alt text](random.png "Title")
## optimal site-selection + optimal site-perturbation
![alt text](optimal.png "Title")

In [42]:
!gdown --fuzzy https://drive.google.com/file/d/1SzrDr9-YcKqkit0KQmcysJQnN4WFMFKJ/view?usp=sharing

Downloading...
From: https://drive.google.com/uc?id=1SzrDr9-YcKqkit0KQmcysJQnN4WFMFKJ
To: /mnt/ufs18/home-107/jiajingh/claw-sat.zip
100%|███████████████████████████████████████| 1.48G/1.48G [00:12<00:00, 116MB/s]


In [38]:
!gdown --fuzzy https://drive.google.com/file/d/1PhwELuRhEtTJEz_sH8PV9Xiq49BU0z7d/view?usp=sharing

Downloading...
From: https://drive.google.com/uc?id=1PhwELuRhEtTJEz_sH8PV9Xiq49BU0z7d
To: /mnt/ufs18/home-107/jiajingh/contracode.zip
100%|█████████████████████████████████████████| 857M/857M [00:07<00:00, 117MB/s]


In [48]:
!rm -r final-models/
!mkdir -p claw-sat/checkpoints
!mkdir -p contracode/checkpoints
!unzip -q claw-sat.zip -d ./claw-sat/checkpoints/Best_F1
!unzip -q contracode.zip -d ./contracode/checkpoints/Best_F1

rm: cannot remove ‘final-models/’: No such file or directory


# CLAW
## Clean example

In [49]:
!python ./Transformer/gradient_attack.py \
    --data_path ./outputs/test.tsv \
    --expt_dir "claw-sat" \
    --load_checkpoint "Best_F1" \
    --save_path ./outputs/targets-test.json \
    --n_alt_iters 2 \
    --z_init 1 \
    --batch_size 32 \
    --u_pgd_epochs 0 \
    --z_epsilon 0 \
    --attack_version 2 \
    --u_learning_rate 0.5 \
    --z_learning_rate 0.5 \
    --smoothing_param 0.01 \
	--vocab_to_use 1 \
    --teacher_ratio 

!python ./Transformer/replace_tokens.py \
      --source_data_path ./outputs/test.tsv \
      --dest_data_path ./outputs/gradient-targeting/test.tsv \
      --mapping_json ./outputs/targets-test-gradient.json

/mnt/home/jiajingh/anaconda3/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: SourceField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/mnt/home/jiajingh/anaconda3/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: TargetField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/mnt/home/jiajingh/anaconda3/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: TransSourceField class will be retired soon and m

In [50]:
!python3 ./Transformer/evaluate.py \
  --data_path ./outputs/gradient-targeting/test.tsv \
  --expt_dir "claw-sat" \
  --output_dir ./outputs \
  --load_checkpoint "Best_F1" \
  --src_field_name "transforms.Combined" --save 

/mnt/home/jiajingh/anaconda3/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: SourceField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/mnt/home/jiajingh/anaconda3/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: TargetField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/mnt/home/jiajingh/anaconda3/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: TransSourceField class will be retired soon and m

## adversarial examples

In [ ]:
!python ./Transformer/gradient_attack.py \
    --data_path ./outputs/test.tsv \
    --expt_dir "claw-sat" \
    --load_checkpoint "Best_F1" \
    --save_path ./outputs/targets-test.json \
    --n_alt_iters 2 \
    --z_init 1 \
    --batch_size 16 \
    --u_pgd_epochs 3 \
    --z_epsilon 5 \
    --attack_version 2 \
    --u_learning_rate 0.5 \
    --z_learning_rate 0.5 \
    --smoothing_param 0.01 \
	--vocab_to_use 1 \
    --teacher_ratio \
    --u_optim \
    --z_optim

!python ./Transformer/replace_tokens.py \
      --source_data_path ./outputs/test.tsv \
      --dest_data_path ./outputs/gradient-targeting/test.tsv \
      --mapping_json ./outputs/targets-test-gradient.json

/mnt/home/jiajingh/anaconda3/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: SourceField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/mnt/home/jiajingh/anaconda3/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: TargetField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/mnt/home/jiajingh/anaconda3/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: TransSourceField class will be retired soon and m

In [34]:
!python3 ./Transformer/evaluate.py \
  --data_path ./outputs/gradient-targeting/test.tsv \
  --expt_dir "claw-sat" \
  --output_dir ./outputs \
  --load_checkpoint "Best_F1" \
  --src_field_name "transforms.Combined" --save 

/mnt/home/jiajingh/anaconda3/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: SourceField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/mnt/home/jiajingh/anaconda3/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: TargetField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/mnt/home/jiajingh/anaconda3/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: TransSourceField class will be retired soon and m

# Contracode
## clean examples

In [8]:
!python ./Transformer/gradient_attack.py \
    --data_path ./outputs/test.tsv \
    --expt_dir "contracode" \
    --load_checkpoint "Best_F1" \
    --save_path ./outputs/targets-test.json \
    --n_alt_iters 2 \
    --z_init 1 \
    --batch_size 32 \
    --u_pgd_epochs 0 \
    --z_epsilon 0 \
    --attack_version 2 \
    --u_learning_rate 0.5 \
    --z_learning_rate 0.5 \
    --smoothing_param 0.01 \
	--vocab_to_use 1 \
    --teacher_ratio 

!python ./Transformer/replace_tokens.py \
      --source_data_path ./outputs/test.tsv \
      --dest_data_path ./outputs/gradient-targeting/test.tsv \
      --mapping_json ./outputs/targets-test-gradient.json

/mnt/home/jiajingh/anaconda3/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: SourceField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/mnt/home/jiajingh/anaconda3/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: TargetField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/mnt/home/jiajingh/anaconda3/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: TransSourceField class will be retired soon and m

In [9]:
!python3 ./Transformer/evaluate.py \
  --data_path ./outputs/gradient-targeting/test.tsv \
  --expt_dir "contracode" \
  --output_dir ./outputs \
  --load_checkpoint "Best_F1" \
  --src_field_name "transforms.Combined" --save 

/mnt/home/jiajingh/anaconda3/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: SourceField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/mnt/home/jiajingh/anaconda3/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: TargetField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/mnt/home/jiajingh/anaconda3/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: TransSourceField class will be retired soon and m

## adversarial examples

In [35]:
!python ./Transformer/gradient_attack.py \
    --data_path ./outputs/test.tsv \
    --expt_dir "contracode" \
    --load_checkpoint "Best_F1" \
    --save_path ./outputs/targets-test.json \
    --n_alt_iters 2 \
    --z_init 1 \
    --batch_size 16 \
    --u_pgd_epochs 3 \
    --z_epsilon 5 \
    --attack_version 2 \
    --u_learning_rate 0.5 \
    --z_learning_rate 0.5 \
    --smoothing_param 0.01 \
	--vocab_to_use 1 \
    --teacher_ratio \
    --u_optim \
    --z_optim

!python ./Transformer/replace_tokens.py \
      --source_data_path ./outputs/test.tsv \
      --dest_data_path ./outputs/gradient-targeting/test.tsv \
      --mapping_json ./outputs/targets-test-gradient.json

/mnt/home/jiajingh/anaconda3/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: SourceField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/mnt/home/jiajingh/anaconda3/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: TargetField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/mnt/home/jiajingh/anaconda3/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: TransSourceField class will be retired soon and m

In [36]:
!python3 ./Transformer/evaluate.py \
  --data_path ./outputs/gradient-targeting/test.tsv \
  --expt_dir "contracode" \
  --output_dir ./outputs \
  --load_checkpoint "Best_F1" \
  --src_field_name "transforms.Combined" --save 

/mnt/home/jiajingh/anaconda3/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: SourceField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/mnt/home/jiajingh/anaconda3/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: TargetField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/mnt/home/jiajingh/anaconda3/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: TransSourceField class will be retired soon and m